<a href="https://colab.research.google.com/github/chw8207/NLP-study/blob/main/ch02_%EB%B9%88%EB%8F%84%EB%B6%84%EC%84%9D_%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 필요한 라이브러리 다운로드

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. matplotlib에 한글 폰트 사용 설정

In [3]:
!apt -qq -y install fonts-nanum

fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
import matplotlib.font_manager as fm
from pathlib import Path
from matplotlib import rc

font_path = Path('/usr/share/fonts/truetype/nanum/NanumGothic.ttf') 
fontprop = fm.FontProperties(fname=font_path)
rc('font', family=fontprop.get_name(), size=16)
# fm._rebuild()

### 3. pandas를 활용하여 엑셀파일 데이터 읽기기

In [5]:
from glob import glob
glob('/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/*')

['/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/spam.csv',
 '/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/영화_위키피디아_최근5년.xlsx']

In [6]:
import pandas as pd
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/'
df = pd.read_excel(ROOT_PATH + '영화_위키피디아_최근5년.xlsx', sheet_name='2021')
df.head()

,순위,영화명,개봉일,개요,줄거리
0,1,스파이더맨: 노 웨이 홈,2021-12-15,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...,전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌...
1,2,모가디슈,2021-07-28,"《모가디슈》는 2021년에 개봉한 대한민국의 영화이다. 류승완 감독의 작품으로, 7...",NaN
2,3,이터널스,2021-11-03,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...,데비안츠라는 괴물들로부터 인류를 지키고 인류를 진화시키기 위해 신과 같은 존재인 셀...
3,4,블랙 위도우,2021-07-07,《블랙 위도우》(영어: Black Widow)는 2021년 개봉한 미국의 슈퍼히어로...,NaN
4,5,분노의 질주: 더 얼티메이트,2021-05-19,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화...",도미닉(빈 디젤)은 자신과 가장 가까웠던 형제 제이콥(존 시나)이 사이퍼(샤를리즈 ...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순위      30 non-null     int64         
 1   영화명     30 non-null     object        
 2   개봉일     30 non-null     datetime64[ns]
 3   개요      30 non-null     object        
 4   줄거리     26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.3+ KB


In [8]:
# 5개년 데이터 가져오기기
df_list = []
for year in ['2021', '2020', '2019', '2018', '2017'] : 
  df_tmp = pd.read_excel(ROOT_PATH + '영화_위키피디아_최근5년.xlsx', sheet_name=year)
  df_list.append(df_tmp)
len(df_list)

5

In [9]:
df_list[4].head()

,순위,영화명,개봉일,개요,줄거리
0,1,택시운전사,2017-08-02,"《택시운전사》는 2017년에 개봉한 대한민국의 영화이다. 실화를 바탕으로 하였으며,...","1980년 5월, 서울에서 택시기사일을 하며 하나 뿐인 딸과 착실하게 살아가는 만섭..."
1,2,신과함께-죄와 벌,2017-12-20,"《신과함께-죄와 벌》은 2017년 12월 20일에 개봉한 대한민국의 영화로, 주호민...",김자홍(차태현)이 삼차사와 49일 동안 7개의 지옥에서 7개의 재판을 받으며 자신의...
2,3,공조,2017-01-18,《공조》는 2017년에 개봉한 대한민국의 영화이다.,특수부대 출신 형사 림철령은 작전수행 도중 지원을 기다리자는 동료들의 말도 무시하고...
3,4,스파이더맨: 홈 커밍,2017-07-05,《스파이더맨: 홈커밍》(영어: Spider-Man: Homecoming)은 마블 코...,-
4,5,범죄도시,2017-10-03,"""범죄도시""는 2017년에 개봉한 대한민국의 영화이다.\n\n서울 구로구 가리봉동 ...","통쾌하고, 화끈하고, 살벌하게 나쁜놈들 때려잡는 강력반 형사들의 ‘조폭소탕작전’ 이..."


In [10]:
# 5개년 데이터 하나로 합치기 
df = pd.concat(df_list)
df.shape

(150, 5)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순위      150 non-null    int64         
 1   영화명     150 non-null    object        
 2   개봉일     150 non-null    datetime64[ns]
 3   개요      150 non-null    object        
 4   줄거리     146 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.0+ KB


In [12]:
# null값 제거하기
# 개요와 줄거리 구분하지 않기
df.dropna(inplace=True)
# drop=True : 이전에 있었던 인덱스가 칼럼으로 들어가는데 사용하지 않음
df.reset_index(drop=True, inplace=True)  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순위      146 non-null    int64         
 1   영화명     146 non-null    object        
 2   개봉일     146 non-null    datetime64[ns]
 3   개요      146 non-null    object        
 4   줄거리     146 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 5.8+ KB


In [13]:
df['text'] = df['개요'] + df['줄거리']
df.head()

,순위,영화명,개봉일,개요,줄거리,text
0,1,스파이더맨: 노 웨이 홈,2021-12-15,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...,전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌...,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...
1,3,이터널스,2021-11-03,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...,데비안츠라는 괴물들로부터 인류를 지키고 인류를 진화시키기 위해 신과 같은 존재인 셀...,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...
2,5,분노의 질주: 더 얼티메이트,2021-05-19,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화...",도미닉(빈 디젤)은 자신과 가장 가까웠던 형제 제이콥(존 시나)이 사이퍼(샤를리즈 ...,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화..."
3,6,싱크홀,2021-08-11,-,-,--
4,7,극장판 귀멸의 칼날: 무한열차편,2021-01-27,《극장판 귀멸의 칼날: 무한열차편》(일본어: 劇場版「鬼滅の刃」無限列車編 영어: De...,혈귀술로 사람들을 잠에 빠뜨리는 오니인 엔무(하현 1)가 무한열차에 나타나 탑승자들...,《극장판 귀멸의 칼날: 무한열차편》(일본어: 劇場版「鬼滅の刃」無限列車編 영어: De...


In [14]:
df['개요'][0]

'《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 2021년 개봉한 미국의 슈퍼히어로 영화이다. 2019년 영화 《스파이더맨: 파 프롬 홈》의 후속작이자, 마블 시네마틱 유니버스(MCU)의 27번째 작품이다. 감독은 전편과 같은 존 왓츠가 맡았으며, 각본 역시 전편에 참여했던 크리스 매케나와 에릭 소머즈가 맡았다.'

In [15]:
df['줄거리'][0]

'전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌우고, 정체를 공개한 장면부터 시작된다. 주변 사람들로부터 관심이 쏠린 피터 파커와 MJ는 웹스윙으로 현장을 벗어나 집으로 가지만, 집 근처에도 많은 사람들이 온다. 이후, 피터, MJ, 네드, 큰엄마 메이는 경찰에 조사받는다. 변호사 맷 머독이 증거불충분으로 기소되지는 않는다고 말하지만 대중들은 스파이더맨을 살인자라며 비난한다.\n\n미스테리오 때문에 자신의 일상이 엉망이 된 피터, 자신이 끌어들인 네드와 MJ까지 논란으로 인해 MIT 진학이 좌절되자 피터는 닥터 스트레인지를 찾아가 과거를 바꿔달라고 하지만 그는 거절한다. 대신 다른 사람들이 피터 파커가 스파이더맨이었다는 기억을 없애주기로 한다. 그러나 피터는 폭로 이전에 자신의 정체를 알던 이들은 예외로 해달라며 주문을 방해하고 스트레인지의 마법은 실패한다. 이로 인해 오히려 스파이더맨의 정체를 알던 평행우주의 인물들이 넘어오게 된다.\n\n피터는 플래시 톰슨의 도움으로 MIT에 한 번 더 기회를 요청하기 위해 공항으로 가던 부총장보를 만나려 한다. 부총장보를 만났지만 갑자기 도로에서 스파이더맨: 트릴로지2의 닥터 옥토퍼스가 나타난다. 피터는 닥터 옥토퍼스를 제압하고 부총장보에게 입학 허가를 받았지만, 2002년 작 스파이더맨의 그린 고블린이 나타난다. 그 때 닥터 스트레인지의 도움으로 생텀으로 온 피터는 닥터 옥토퍼스와 함께 잡혀 있던 어메이징 스파이더맨의 리저드도 보게 된다. 스트레인지는 평행우주의 인물들을 막으려 했지만 일부가 우리 차원으로 넘어왔다고 말하며 피터에게 사건 해결을 촉구한다.\n\n네드와 MJ의 도움으로 스파이더맨 3의 샌드맨과 어메이징 스파이더맨 2의 일렉트로를 만나고 둘을 모두 잡게 된다. 사악한 그린 고블린의 인격에서 잠시 벗어난 노먼 오스본은 메이의 일터로 가고, 피터와 함께 생텀의 지하로 간다. 닥터 스트레인지는 그들을 되돌려 보내려고 하지만, 죽을 운명이었던 스파이더맨의 숙적들을 지키기 위해 

In [16]:
df['text'][0]

'《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 2021년 개봉한 미국의 슈퍼히어로 영화이다. 2019년 영화 《스파이더맨: 파 프롬 홈》의 후속작이자, 마블 시네마틱 유니버스(MCU)의 27번째 작품이다. 감독은 전편과 같은 존 왓츠가 맡았으며, 각본 역시 전편에 참여했던 크리스 매케나와 에릭 소머즈가 맡았다.전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌우고, 정체를 공개한 장면부터 시작된다. 주변 사람들로부터 관심이 쏠린 피터 파커와 MJ는 웹스윙으로 현장을 벗어나 집으로 가지만, 집 근처에도 많은 사람들이 온다. 이후, 피터, MJ, 네드, 큰엄마 메이는 경찰에 조사받는다. 변호사 맷 머독이 증거불충분으로 기소되지는 않는다고 말하지만 대중들은 스파이더맨을 살인자라며 비난한다.\n\n미스테리오 때문에 자신의 일상이 엉망이 된 피터, 자신이 끌어들인 네드와 MJ까지 논란으로 인해 MIT 진학이 좌절되자 피터는 닥터 스트레인지를 찾아가 과거를 바꿔달라고 하지만 그는 거절한다. 대신 다른 사람들이 피터 파커가 스파이더맨이었다는 기억을 없애주기로 한다. 그러나 피터는 폭로 이전에 자신의 정체를 알던 이들은 예외로 해달라며 주문을 방해하고 스트레인지의 마법은 실패한다. 이로 인해 오히려 스파이더맨의 정체를 알던 평행우주의 인물들이 넘어오게 된다.\n\n피터는 플래시 톰슨의 도움으로 MIT에 한 번 더 기회를 요청하기 위해 공항으로 가던 부총장보를 만나려 한다. 부총장보를 만났지만 갑자기 도로에서 스파이더맨: 트릴로지2의 닥터 옥토퍼스가 나타난다. 피터는 닥터 옥토퍼스를 제압하고 부총장보에게 입학 허가를 받았지만, 2002년 작 스파이더맨의 그린 고블린이 나타난다. 그 때 닥터 스트레인지의 도움으로 생텀으로 온 피터는 닥터 옥토퍼스와 함께 잡혀 있던 어메이징 스파이더맨의 리저드도 보게 된다. 스트레인지는 평행우주의 인물들을 막으려 했지만 일부가 우리 차원으로 넘어왔다고 말하며 피터에게 사건 해결

### 4. 분리하지 않기 원하는 단어 설정

In [17]:
from konlpy.tag import Okt
okt = Okt()
# pos(part of speech) : 품사를 할당해주는 함수
okt.pos('닥터 스트레인지는 스파이더맨에 나온다.')

[('닥터', 'Noun'),
 ('스트레인지', 'Noun'),
 ('는', 'Josa'),
 ('스파이더맨', 'Noun'),
 ('에', 'Josa'),
 ('나온다', 'Verb'),
 ('.', 'Punctuation')]

In [20]:
# 경로 확인
import konlpy
konlpy.__path__

['/usr/local/lib/python3.9/dist-packages/konlpy']

In [21]:
import os
os.chdir('/usr/local/lib/python3.9/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.9/dist-packages/konlpy/java'

In [22]:
# 임시 폴더 생성
!mkdir tmp

In [23]:
import os
os.chdir('/usr/local/lib/python3.9/dist-packages/konlpy/java/tmp')
os.getcwd()

'/usr/local/lib/python3.9/dist-packages/konlpy/java/tmp'

In [24]:
# 상위 폴더 확인하기
!ls ../

aho-corasick.jar     kkma-2.0.jar		 shineware-ds-1.0.jar
bin		     komoran-3.0.jar		 snakeyaml-1.12.jar
conf		     open-korean-text-2.1.0.jar  tmp
data		     scala-library-2.12.3.jar	 twitter-text-1.14.7.jar
jhannanum-0.8.4.jar  shineware-common-1.0.jar


In [25]:
!jar xvf ../open-korean-text-2.1.0.jar

  created: META-INF/
 inflated: META-INF/MANIFEST.MF
  created: org/
  created: org/openkoreantext/
  created: org/openkoreantext/processor/
  created: org/openkoreantext/processor/normalizer/
  created: org/openkoreantext/processor/phrase_extractor/
  created: org/openkoreantext/processor/qa/
  created: org/openkoreantext/processor/stemmer/
  created: org/openkoreantext/processor/tokenizer/
  created: org/openkoreantext/processor/tools/
  created: org/openkoreantext/processor/util/
  created: org/openkoreantext/processor/util/adjective/
  created: org/openkoreantext/processor/util/adverb/
  created: org/openkoreantext/processor/util/auxiliary/
  created: org/openkoreantext/processor/util/freq/
  created: org/openkoreantext/processor/util/josa/
  created: org/openkoreantext/processor/util/noun/
  created: org/openkoreantext/processor/util/substantives/
  created: org/openkoreantext/processor/util/typos/
  created: org/openkoreantext/processor/util/verb/
 inflated: org/openkoreantext/pr

In [26]:
with open('/usr/local/lib/python3.9/dist-packages/konlpy/java/tmp/org/openkoreantext/processor/util/noun/names.txt') as f :
  names = f.read()
names

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n'

In [27]:
names += '스트레인지\n'

In [28]:
with open('/usr/local/lib/python3.9/dist-packages/konlpy/java/tmp/org/openkoreantext/processor/util/noun/names.txt', 'w') as f :
  f.write(names)

In [29]:
names

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n스트레인지\n'

In [30]:
import os
os.getcwd()

'/usr/local/lib/python3.9/dist-packages/konlpy/java/tmp'

In [31]:
# tmp변경 내용을 jar파일에 넣어주는 작업
!jar cvf ../open-korean-text-2.1.0.jar *

added manifest
ignoring entry META-INF/
ignoring entry META-INF/MANIFEST.MF
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties(in = 119) (out= 110)(deflated 7%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml(in = 9127) (out= 2208)(deflated 75%)
adding: org/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/qa/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/qa/BatchGetUnknownNouns$ChunkWithTweet$.class(in = 2376) (out= 952)(deflated 59%)
adding: org/openkoreantext/processor/qa/BatchGetUnknownNouns$ChunkWithTweet.class(in = 3080) (out= 1338)(deflated 56%)
adding: org/openkoreantext/proc

In [32]:
# from konlpy.tag import Okt
# okt = Okt()
# # pos(part of speech) : 품사를 할당해주는 함수
okt.pos('닥터 스트레인지는 스파이더맨에 나온다.')

[('닥터', 'Noun'),
 ('스', 'Noun'),
 ('트레인', 'Noun'),
 ('지는', 'Josa'),
 ('스파이더맨', 'Noun'),
 ('에', 'Josa'),
 ('나온다', 'Verb'),
 ('.', 'Punctuation')]

### 5. 영화 한 개에 대한 빈도분석과 워드 클라우드

In [37]:
one_text = df['text'][0]
one_text

'《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 2021년 개봉한 미국의 슈퍼히어로 영화이다. 2019년 영화 《스파이더맨: 파 프롬 홈》의 후속작이자, 마블 시네마틱 유니버스(MCU)의 27번째 작품이다. 감독은 전편과 같은 존 왓츠가 맡았으며, 각본 역시 전편에 참여했던 크리스 매케나와 에릭 소머즈가 맡았다.전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌우고, 정체를 공개한 장면부터 시작된다. 주변 사람들로부터 관심이 쏠린 피터 파커와 MJ는 웹스윙으로 현장을 벗어나 집으로 가지만, 집 근처에도 많은 사람들이 온다. 이후, 피터, MJ, 네드, 큰엄마 메이는 경찰에 조사받는다. 변호사 맷 머독이 증거불충분으로 기소되지는 않는다고 말하지만 대중들은 스파이더맨을 살인자라며 비난한다.\n\n미스테리오 때문에 자신의 일상이 엉망이 된 피터, 자신이 끌어들인 네드와 MJ까지 논란으로 인해 MIT 진학이 좌절되자 피터는 닥터 스트레인지를 찾아가 과거를 바꿔달라고 하지만 그는 거절한다. 대신 다른 사람들이 피터 파커가 스파이더맨이었다는 기억을 없애주기로 한다. 그러나 피터는 폭로 이전에 자신의 정체를 알던 이들은 예외로 해달라며 주문을 방해하고 스트레인지의 마법은 실패한다. 이로 인해 오히려 스파이더맨의 정체를 알던 평행우주의 인물들이 넘어오게 된다.\n\n피터는 플래시 톰슨의 도움으로 MIT에 한 번 더 기회를 요청하기 위해 공항으로 가던 부총장보를 만나려 한다. 부총장보를 만났지만 갑자기 도로에서 스파이더맨: 트릴로지2의 닥터 옥토퍼스가 나타난다. 피터는 닥터 옥토퍼스를 제압하고 부총장보에게 입학 허가를 받았지만, 2002년 작 스파이더맨의 그린 고블린이 나타난다. 그 때 닥터 스트레인지의 도움으로 생텀으로 온 피터는 닥터 옥토퍼스와 함께 잡혀 있던 어메이징 스파이더맨의 리저드도 보게 된다. 스트레인지는 평행우주의 인물들을 막으려 했지만 일부가 우리 차원으로 넘어왔다고 말하며 피터에게 사건 해결

In [38]:
one_text_tokenized = okt.morphs(one_text)
one_text_tokenized[:12]

['《', '스파이더맨', ':', '노', '웨이', '홈', '》', '(', '영어', ':', 'Spider', '-']

In [39]:
one_text_tokenized = okt.nouns(one_text)
one_text_tokenized[:12]

['스파이더맨', '노', '웨이', '홈', '영어', '은', '개봉', '미국', '슈퍼히어로', '영화', '영화', '스파이더맨']

In [40]:
df_word_count = pd.DataFrame()
word_cloud_dict = {}  # key:word, value:word_count

word_list = list(set(one_text_tokenized))  # set 사용하면 unique한 단어들만 뽑을 수 있음
word_cnt_list = []

In [41]:
from tqdm import tqdm

for word in tqdm(word_list) : 
  word_cnt = one_text_tokenized.count(word)
  word_cnt_list.append(word_cnt)
  if len(word) == 1 : # 1개의 단어 불용어 처리
    continue
  word_cloud_dict[word] = word_cnt

100%|██████████| 252/252 [00:00<00:00, 45694.72it/s]


In [42]:
df_word_count['word'] = word_list
df_word_count['word_cnt'] = word_cnt_list
df_word_count = df_word_count.sort_values('word_cnt', ascending=False) 
df_word_count.head(30)

,word,word_cnt
152,피터,34
123,스파이더맨,29
162,스트레인지,12
78,닥터,11
94,그린,9
45,그,9
44,말,9
103,고블린,9
211,의,8
231,자신,8


In [ ]:
# 워드클라우드 만들기
from wordcloud import WordCloud

word_cloud = WordCloud(font_path=font_path, width=1000, height=600)
word_cloud.generate_from_frequencies(word_cloud_dict)

In [45]:
word_cloud_dict

{'고립': 1,
 '슈트': 1,
 '누명': 1,
 '공개': 1,
 '도로': 1,
 '우리': 1,
 '이전': 1,
 '마법': 4,
 '증거불충분': 1,
 '촉구': 1,
 '인격': 1,
 '슬링': 3,
 '토비': 3,
 '이후': 1,
 '상자': 1,
 '조사': 1,
 '도덕': 1,
 '감독': 1,
 '위해': 4,
 '시리즈': 1,
 '치료': 4,
 '거절': 1,
 '해결': 2,
 '논란': 1,
 '의무': 1,
 '방해': 1,
 '결과': 1,
 '앤드류': 1,
 '살인자': 2,
 '가득': 1,
 '차원': 2,
 '지하': 1,
 '일리': 2,
 '혼란': 1,
 '케나': 1,
 '도피': 1,
 '각본': 1,
 '뷰글': 2,
 '상황': 1,
 '시네마': 1,
 '진실': 1,
 '평행우주': 2,
 '기계': 1,
 '불청객': 1,
 '연구': 1,
 '예외': 1,
 '가필드': 1,
 '무덤': 1,
 '오지': 1,
 '다른': 4,
 '등장': 1,
 '계획': 1,
 '마블': 1,
 '로부터': 2,
 '갑자기': 1,
 '해방': 1,
 '닥터': 11,
 '이로': 1,
 '운명': 1,
 '호건': 1,
 '공간': 1,
 '진행': 1,
 '좌절': 1,
 '이상': 2,
 '오스본': 1,
 '이방인': 2,
 '버전': 2,
 '그린': 9,
 '기소': 1,
 '대중': 1,
 '역시': 1,
 '파커': 6,
 '폭로': 1,
 '비난': 2,
 '찾기': 1,
 '고블린': 9,
 '원래': 2,
 '머독': 1,
 '플래시': 1,
 '노먼': 1,
 '잠시': 1,
 '근처': 1,
 '모두': 3,
 '때문': 1,
 '장보': 3,
 '제압': 1,
 '옥토': 5,
 '정체': 3,
 '생텀': 2,
 '기회': 1,
 '대신': 1,
 '정신': 1,
 '톰슨': 1,
 '스파이더맨': 29,
 '약속': 1,
 '사건': 1,
 '어메이징': 3,
 '썩힌다'

### 6. 여러 개의 영화별 빈도 분석과 워드 클라우드 만들기

In [46]:
df.shape

(146, 6)

In [48]:
df_87 = df[(df['text'].apply(lambda x: len(x)) > 100)]
df_87.reset_index(drop=True, inplace=True)
df_87.shape

(87, 6)

In [49]:
df_87.head()

,순위,영화명,개봉일,개요,줄거리,text
0,1,스파이더맨: 노 웨이 홈,2021-12-15,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...,전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌...,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...
1,3,이터널스,2021-11-03,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...,데비안츠라는 괴물들로부터 인류를 지키고 인류를 진화시키기 위해 신과 같은 존재인 셀...,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...
2,5,분노의 질주: 더 얼티메이트,2021-05-19,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화...",도미닉(빈 디젤)은 자신과 가장 가까웠던 형제 제이콥(존 시나)이 사이퍼(샤를리즈 ...,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화..."
3,7,극장판 귀멸의 칼날: 무한열차편,2021-01-27,《극장판 귀멸의 칼날: 무한열차편》(일본어: 劇場版「鬼滅の刃」無限列車編 영어: De...,혈귀술로 사람들을 잠에 빠뜨리는 오니인 엔무(하현 1)가 무한열차에 나타나 탑승자들...,《극장판 귀멸의 칼날: 무한열차편》(일본어: 劇場版「鬼滅の刃」無限列車編 영어: De...
4,9,소울,2021-01-20,《소울》(영어: Soul)은 2020년 개봉한 미국의 판타지 애니메이션 영화이다. ...,조 가드너는 뉴욕에서 중학교 음악 선생님으로 일한다. 어린 시절 아버지를 따라 피아...,《소울》(영어: Soul)은 2020년 개봉한 미국의 판타지 애니메이션 영화이다. ...


In [50]:
tokens_list = [] 
for i in range(len(df_87)) : 
  tokens = okt.nouns(df_87['text'][i])
  tokens_list.append(tokens)
len(tokens_list)

87

In [51]:
tokens_list[0]

['스파이더맨',
 '노',
 '웨이',
 '홈',
 '영어',
 '은',
 '개봉',
 '미국',
 '슈퍼히어로',
 '영화',
 '영화',
 '스파이더맨',
 '파',
 '프롬',
 '홈',
 '의',
 '후속작',
 '이자',
 '마블',
 '시네마',
 '유니버스',
 '의',
 '작품',
 '감독',
 '전편',
 '존',
 '왓츠',
 '각본',
 '역시',
 '전편',
 '참여',
 '크리스',
 '케나',
 '에릭',
 '소머즈',
 '전작',
 '스파이더맨',
 '파',
 '프롬',
 '홈',
 '미스테리',
 '스파이더맨',
 '살인자',
 '누명',
 '정체',
 '공개',
 '장면',
 '시작',
 '주변',
 '사람',
 '로부터',
 '관심',
 '피터',
 '파커',
 '웹',
 '스윙',
 '현장',
 '집',
 '가지',
 '집',
 '근처',
 '사람',
 '이후',
 '피터',
 '네드',
 '엄마',
 '메이',
 '경찰',
 '조사',
 '변호사',
 '맷',
 '머독',
 '증거불충분',
 '기소',
 '말',
 '대중',
 '스파이더맨',
 '살인자',
 '라며',
 '비난',
 '미스테리',
 '오',
 '때문',
 '자신',
 '일상',
 '엉망',
 '피터',
 '자신',
 '네드',
 '논란',
 '진학',
 '좌절',
 '피터',
 '닥터',
 '스트레인지',
 '과거',
 '달라',
 '그',
 '거절',
 '대신',
 '다른',
 '사람',
 '피터',
 '파커',
 '스파이더맨',
 '기억',
 '피터',
 '폭로',
 '이전',
 '자신',
 '정체',
 '이',
 '예외',
 '주문',
 '방해',
 '스트레인지',
 '마법',
 '이로',
 '오히려',
 '스파이더맨',
 '정체',
 '평행우주',
 '인물',
 '피터',
 '플래시',
 '톰슨',
 '도움',
 '번',
 '더',
 '기회',
 '요청',
 '위해',
 '공항',
 '부총',
 '장보',
 '부총',
 '장보',
 '갑자기',
 '

In [59]:
def get_word_count(tokens) : 
  df_word_count = pd.DataFrame()
  word_list = list(set(tokens))

  word_cnt_list = []
  word_cloud_dict = {}
  for word in word_list : 
    word_cnt = tokens.count(word)
    word_cnt_list.append(word_cnt)
    if (len(word) == 1) | (word in ['영화','개봉']) : 
      continue
    word_cloud_dict[word] = word_cnt

  df_word_count['word'] = word_list
  df_word_count['word_cnt'] = word_cnt_list
  df_word_count = df_word_count.sort_values('word_cnt', ascending=False)

  return word_cloud_dict, df_word_count

In [60]:
def save_word_cloud(word_cloud_dict, save_path) : 
  word_cloud = WordCloud(font_path=font_path, width=1000, height=600)
  word_cloud.generate_from_frequencies(word_cloud_dict)
  word_cloud.to_file(save_path)

In [65]:
for i, tokens in enumerate(tqdm(tokens_list)) : 
  word_cloud_dict, df_word_count = get_word_count(tokens)
  movie_name = str(df_87['영화명'][i]).replace(':', ' ').replace('.', '')
  df_word_count.to_csv(ROOT_PATH + movie_name + '.csv')
  save_path = ROOT_PATH + movie_name + '.png'
  # save_word_cloud(word_cloud_dict, save_path)

100%|██████████| 87/87 [00:02<00:00, 43.02it/s]


In [ ]:
ROOT_PATH